# Building our First RAG bot - Skill: talk to Search Engine

We have now all the building blocks to build our first Bot that "talks with my data". These blocks are:

1) A well indexed hybrid (text and vector) engine with my data in chunks -> Azure AI Search
2) A good LLM python framework to build LLM Apps -> LangChain
3) Quality OpenAI GPT models that understand language and follow instructions -> GPT3.5 and GPT4
4) A persisten memory database -> CosmosDB

We are missing just one thing: **Agents**.

In this Notebook we introduce the concept of Agents and we use it to build or first RAG bot.

In [2]:
import random
import asyncio
from typing import Dict, List
from concurrent.futures import ThreadPoolExecutor
from typing import Optional, Type

from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import AzureChatOpenAI
from langchain_core.runnables import ConfigurableField, ConfigurableFieldSpec
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory, CosmosDBChatMessageHistory
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

#custom libraries that we will use later in the app
from common.utils import  GetDocSearchResults_Tool
from common.prompts import AGENT_DOCSEARCH_PROMPT

from IPython.display import Markdown, HTML, display  

def printmd(string):
    display(Markdown(string))

from dotenv import load_dotenv
load_dotenv("credentials.env")


True

In [3]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

## Introducing: Agents

The implementation of Agents is inspired by two papers: the [MRKL Systems](https://arxiv.org/abs/2205.00445) paper (pronounced ‘miracle’ 😉) and the [ReAct](https://arxiv.org/abs/2210.03629) paper.

Agents are a way to leverage the ability of LLMs to understand and act on prompts. In essence, an Agent is an LLM that has been given a very clever initial prompt. The prompt tells the LLM to break down the process of answering a complex query into a sequence of steps that are resolved one at a time.

Agents become really cool when we combine them with ‘experts’, introduced in the MRKL paper. Simple example: an Agent might not have the inherent capability to reliably perform mathematical calculations by itself. However, we can introduce an expert - in this case a calculator, an expert at mathematical calculations. Now, when we need to perform a calculation, the Agent can call in the expert rather than trying to predict the result itself. This is actually the concept behind [ChatGPT Pluggins](https://openai.com/blog/chatgpt-plugins).

In our case, in order to solve the problem "How do I build a smart bot that talks to my data", we need this REACT/MRKL approach, in which we need to instruct the LLM that it needs to use 'experts/tools' in order to read/load/understand/interact with a any particular source of data.

Let's create then an Agent that interact with the user and uses a Tool to get the information from the Search engine.

#### We start first defining the Tool/Expert

In [4]:
index1_name = "cogsrch-index-files"
index2_name = "cogsrch-index-csv"
index3_name = "cogsrch-index-books"
indexes = [index1_name, index2_name, index3_name]

We have to convert the Retreiver object into a Tool object ("the expert"). Check out the Tool `GetDocSearchResults_Tool` in `utils.py`

Declare the tools the agent will use

In [5]:
topK=7
tools = [GetDocSearchResults_Tool(indexes=indexes, k=5, reranker_th=1, sas_token=os.environ['BLOB_SAS_TOKEN'])]

Get the prompt to use `AGENT_DOCSEARCH_PROMPT` - you can modify this in `prompts.py`! Check it out!

In [6]:
prompt = AGENT_DOCSEARCH_PROMPT

Define the LLM to use

In [7]:
COMPLETION_TOKENS = 1500
llm = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], temperature=0.5, max_tokens=COMPLETION_TOKENS, streaming=True).configurable_alternatives(
    ConfigurableField(id="model"),
    default_key="gpt35",
    gpt4=AzureChatOpenAI(deployment_name=os.environ["GPT4_DEPLOYMENT_NAME"], temperature=0.5, max_tokens=COMPLETION_TOKENS, streaming=True),
)

Construct the OpenAI Tools agent.
> OpenAI API has deprecated functions in favor of tools. The difference between the two is that the tools API allows the model to request that multiple functions be invoked at once, which can reduce response times in some architectures. It’s recommended to use the tools agent for OpenAI models.

In [8]:
agent = create_openai_tools_agent(llm.with_config(configurable={"model": "gpt35"}), tools, prompt)

Create an agent executor by passing in the agent and tools

In [9]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

Give it memory - since AgentExecutor is also a Runnable class, we do the same with did on Notebook 5

In [10]:
def get_session_history(session_id: str, user_id: str) -> CosmosDBChatMessageHistory:
    cosmos = CosmosDBChatMessageHistory(
        cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
        cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
        cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
        connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
        session_id=session_id,
        user_id=user_id
        )

    # prepare the cosmosdb instance
    cosmos.prepare_cosmos()
    return cosmos

Because cosmosDB needs two fields (an id and a partition), and RunnableWithMessageHistory takes by default only one identifier for memory (session_id), we need to use `history_factory_config` parameter and define the multiple keys for the memory class

In [11]:
userid_spec = ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        )
session_id = ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        )

In [12]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
    history_factory_config=[userid_spec,session_id]
)

In [13]:
# configure the session id and user id
random_session_id = "session"+ str(random.randint(1, 1000))
ramdom_user_id = "user"+ str(random.randint(1, 1000))

config={"configurable": {"session_id": random_session_id, "user_id": ramdom_user_id}}
config

{'configurable': {'session_id': 'session469', 'user_id': 'user580'}}

Run the Agent!

In [14]:
%%time
agent_with_chat_history.invoke({"question": "Hi, I'm Pablo Marin. What's yours"}, config=config)

CPU times: user 196 ms, sys: 10.3 ms, total: 206 ms
Wall time: 4.42 s


{'question': "Hi, I'm Pablo Marin. What's yours",
 'history': [],
 'output': "I'm Jarvis, your assistant. How can I help you today, Pablo Marin?"}

In [15]:
printmd(agent_with_chat_history.invoke(
    {"question": "What are markov chains and is there an application in medicine?"}, 
    config=config)["output"])

**Markov Chains:**

Markov chains are mathematical systems that transition from one state to another in a probabilistic manner. The transition from the current state to the next state depends only on the current state and not on the sequence of events that preceded it. This memoryless property makes Markov chains a powerful tool for modeling stochastic processes where the future state is determined by the present state alone.

**Application in Medicine:**

Markov chains find various applications in medicine, particularly in modeling disease progression, treatment outcomes, and healthcare resource allocation. Here are some key applications:

1. **Disease Progression Modeling:** Markov chains are used to model the progression of diseases over time, considering different states such as healthy, diseased, or recovered. These models help in predicting the likelihood of transitioning between states and estimating the impact of interventions or treatments on disease progression<sup><a href="https://medicalresearch.org/article1.pdf?s=diseaseprogression&category=modeling&sort=asc&page=1" target="_blank">[1]</a></sup>.

2. **Treatment Outcomes:** In healthcare, Markov models are employed to simulate the long-term effects of different treatment strategies on patient outcomes. By incorporating probabilities of treatment success, relapse, or adverse events, these models aid in decision-making processes for selecting optimal treatment pathways<sup><a href="https://healthcareanalysis.org/article2.html?s=treatmentoutcomes&category=modeling&sort=asc" target="_blank">[2]</a></sup>.

3. **Healthcare Resource Allocation:** Markov chains assist in optimizing resource allocation within healthcare systems. By analyzing patient flows, disease prevalence, and treatment effectiveness, these models help in determining the most efficient allocation of healthcare resources such as hospital beds, medical staff, and equipment<sup><a href="https://healthcaremanagement.org/article3.csv?s=resourceallocation&category=optimization&sort=asc&page=1" target="_blank">[3]</a></sup>.

4. **Clinical Decision Support:** Markov models are integrated into clinical decision support systems to provide evidence-based recommendations for patient care. These models consider patient-specific factors, treatment histories, and predicted outcomes to assist healthcare professionals in making informed decisions<sup><a href="https://clinicalresearch.org/article4.pdf?s=clinicaldecisionsupport&category=healthcare&sort=asc&page=1" target="_blank">[4]</a></sup>.

By leveraging the probabilistic nature of Markov chains, the healthcare industry can enhance decision-making processes, improve patient outcomes, and optimize resource utilization in medical settings.

In [16]:
try:
    printmd(agent_with_chat_history.invoke(
        {"question": "Interesting, Tell me more about the use specifically in the spread of viruses"},
        config=config)["output"])
except Exception as e:
    print(e)

**Markov Chains in Virus Spread Modeling:**

Markov chains play a crucial role in modeling the spread of viruses and infectious diseases. By capturing the stochastic nature of transmission dynamics, Markov models provide valuable insights into the progression of epidemics, the effectiveness of control measures, and the impact of interventions. Here are some key aspects of using Markov chains in virus spread modeling:

1. **Transmission Dynamics:** Markov chains can represent the transmission dynamics of viruses within populations by defining different states such as susceptible, infected, and recovered. The transitions between these states are governed by transition probabilities that account for factors like contact rates, transmission probabilities, and recovery rates<sup><a href="https://virusresearch.org/article5.pdf?s=transmissiondynamics&category=modeling&sort=asc&page=1" target="_blank">[5]</a></sup>.

2. **Epidemic Forecasting:** Markov models are utilized to forecast the spread of viruses and predict the potential size and duration of epidemics. By simulating multiple scenarios based on different parameters and assumptions, these models help in estimating the trajectory of an outbreak and assessing the effectiveness of various containment strategies<sup><a href="https://epidemiologystudy.org/article6.html?s=epidemicforecasting&category=modeling&sort=asc" target="_blank">[6]</a></sup>.

3. **Effectiveness of Interventions:** Markov chains enable researchers to evaluate the impact of interventions such as vaccination campaigns, social distancing measures, and quarantine protocols on controlling virus spread. By adjusting transition probabilities to reflect the effects of interventions, these models provide insights into the effectiveness of different control strategies<sup><a href="https://publichealthanalysis.org/article7.csv?s=interventioneffectiveness&category=controlstrategies&sort=asc&page=1" target="_blank">[7]</a></sup>.

4. **Spatial and Temporal Analysis:** Markov models can incorporate spatial and temporal dimensions to analyze how viruses spread across different regions and evolve over time. By considering movement patterns, population densities, and environmental factors, these models offer a comprehensive understanding of the spatial dynamics of virus transmission<sup><a href="https://spatialanalysis.org/article8.pdf?s=spatialtemporalanalysis&category=virustransmission&sort=asc&page=1" target="_blank">[8]</a></sup>.

Through the application of Markov chains in virus spread modeling, epidemiologists and public health officials can make informed decisions regarding disease control strategies, resource allocation, and outbreak management to mitigate the impact of infectious diseases on populations.

In [17]:
printmd(agent_with_chat_history.invoke({"question": "Thhank you!"}, config=config)["output"])

You're welcome! If you have any more questions in the future, feel free to ask. Have a great day, Pablo Marin!

#### Important: there is a limitation of GPT3.5, once we start adding long prompts, and long contexts and thorough answers, or the agent makes multiple searches for multi-step questions, we run out of space!

You can minimize this by:
- Shorter System Prompt
- Smaller chunks (less than the default of 5000 characters)
- Reducing topK to bring less relevant chunks

However, you ultimately are sacrificing quality to make everything work with GPT3.5 (cheaper and faster model)

### Let's add more things we have learned so far: dynamic LLM selection of GPT4 and asyncronous streaming

In [18]:
agent = create_openai_tools_agent(llm.with_config(configurable={"model": "gpt4"}), tools, prompt) # We select now GPT-4
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)
agent_with_chat_history = RunnableWithMessageHistory(agent_executor,get_session_history,input_messages_key="question", 
                                                     history_messages_key="history", history_factory_config=[userid_spec,session_id])

In prior notebooks with use the function `.stream()` of the runnable in order to stream the tokens. However if you need to stream individual tokens from the agent or surface steps occuring within tools, you would need to use a combination of `Callbacks` and `.astream()` OR the new `astream_events` API (beta).

Let’s use here the astream_events API to stream the following events:

    Agent Start with inputs
    Tool Start with inputs
    Tool End with outputs
    Stream the agent final anwer token by token
    Agent End with outputs

In [19]:
QUESTION = "Tell me more about your last answer, search again multiple times and provide a deeper explanation"

In [20]:
async for event in agent_with_chat_history.astream_events(
    {"question": QUESTION}, config=config, version="v1",
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "AgentExecutor"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "AgentExecutor"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        # print(f"Tool output was: {event['data'].get('output')}")
        print("--")

Starting agent: AgentExecutor
--
Starting tool: docsearch with inputs: {'query': 'Markov chains virus spread modeling'}
--
Starting tool: docsearch with inputs: {'query': 'Markov chains epidemic forecasting'}
--
Starting tool: docsearch with inputs: {'query': 'Markov chains spatial temporal analysis in virus transmission'}
--
Starting tool: docsearch with inputs: {'query': 'Markov chains effectiveness of interventions in virus spread'}
Done tool: docsearch
--
Done tool: docsearch
--
Done tool: docsearch
--
Done tool: docsearch
--
Based on the information from the documents, Markov chains are used in various ways to model the spread of viruses and analyze the effectiveness of interventions. Here is a deeper explanation of their use in these contexts:

1. **Spatial Markov Chain Models for Virus Spread**:
    - A Spatial Markov Chain model represents the spread of viruses by using a graph structure where nodes represent individuals, and edges represent relationships or contacts between th

#### Note: Try to run this last question with GPT3.5 and see how you are going to run out of token space in the LLM

# Summary

We just built our first RAG BOT!.

- We learned that **Agents + Tools are the best way to go about building Bots**. <br>
- We converted the Azure Search retriever into a Tool using the function `GetDocSearchResults_Tool` in `utils.py`
- We learned about the events API (Beta), one way to stream the answer from agents
- We learned that for comprehensive, quality answers we will run out of space with GPT3.5. GPT4 then becomes necessary.


# NEXT

Now that we have a bot with one skill (Document Search), let's build more skills!. In the next Notebook, we are going to build an agent that can understand tabular data in csv file and can execute python commands